In [1]:
# Import Python package
import BinhoSupernova
from BinhoSupernova.Supernova import Supernova
from BinhoSupernova.commands.definitions import *

## Getting started

#### 1. List all the Supernova devices connected to the PC host

The ``BinhoSupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

In [2]:
BinhoSupernova.getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#7&f321146&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': '59314AC593A1C05C8163B6759994476',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': 0}]

#### 2. Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [3]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
supernova = Supernova()

#### 3. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path, logEnable)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoSupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.
- ``logEnable``: A boolean flag to enable the logging of messages. Log messages are saved in a system directory based on the OS.


In [4]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [5]:
import json

# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:
    
    if supernova_message != None:

        # Print a header
        print(">> New message from SUPERNOVA:")

        # If the command is GET TARGET DEVICE TABLE, convert to json.
        if supernova_message["name"] == "I3C GET TARGET DEVICE TABLE" or supernova_message["name"] == "I3C TRANSFER":
            responseJson = json.dumps(supernova_message, indent=4)
            print(responseJson)

        else:
            print(supernova_message)
    
    if system_message != None:

        # Print a header
        print(">> New message from the SYSTEM:")

        print(system_message)

In [6]:
# Register callback function
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[0, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

#### 6. Test communication with a basic command such as ``GET USB STRING``

The ``GET USB STRING`` command allows you to retrieve various string values related to the product, such as the manufacturer, product name, serial number, firmware version, and hardware version. This command serves as a basic test to verify the communication and functionality with the Supernova device. By successfully receiving the expected string values, you can ensure that the communication between the PC host and the Supernova device is functioning as expected.

In [9]:
# GET USB STRING - Get Manufacturer
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.MANUFACTURER)

# GET USB STRING  - Get Product Name
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.PRODUCT_NAME)

# GET USB STRING  - Get Serial Number
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.SERIAL_NUMBER)

# GET USB STRING  - Firmware Version
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.FW_VERSION)

# GET USB STRING  - Hardware Version
request_result = supernova.getUsbString(getId(), GetUsbStringSubCommand.HW_VERSION)

>> New message from SUPERNOVA:
{'id': 2, 'command': 96, 'name': 'GET USB STRING', 'length': 12, 'message': 'MN-Binho LLC'}
>> New message from SUPERNOVA:
{'id': 3, 'command': 96, 'name': 'GET USB STRING', 'length': 18, 'message': 'PR-Binho Supernova'}
>> New message from SUPERNOVA:
{'id': 4, 'command': 96, 'name': 'GET USB STRING', 'length': 35, 'message': 'SN-59314AC593A1C05C8163B6759994476D'}
>> New message from SUPERNOVA:
{'id': 5, 'command': 96, 'name': 'GET USB STRING', 'length': 19, 'message': 'FW-2.3.0-45-b3f852f'}
>> New message from SUPERNOVA:
{'id': 6, 'command': 96, 'name': 'GET USB STRING', 'length': 4, 'message': 'HW-B'}


## I3C Protocol API

The I3C Protocol API methods are described below.

### 1. Configure the Supernova as an I3C Controller

In [9]:
supernova.i3cControllerInit(getId())

{'module': 1,
 'opcode': 0,
 'message': 'INITIALIZE I3C CONTROLLER requests success'}

>> New message from SUPERNOVA:
{'id': 6, 'command': 1, 'name': 'I3C CONTROLLER INIT', 'result': 'I3C_CONTROLLER_INIT_SUCCESS'}


### 2. Set I3C bus voltage

There are 2 I3C ports:
- I3C LV: from 800 mV to 1199 mV
- I3C HV: from 1200 mV to 3300 mV

In [10]:
request_result = supernova.setI3cBusVoltage(getId(), 3300)

>> New message from SUPERNOVA:
{'id': 7, 'command': 97, 'name': 'SET I3C BUS VOLTAGE', 'result': 0}


### 3. I3C bus initialization

There are two options:
1. Initialize the bus with an **empty target devices table** and scan the I3C bus.

In this case, the I3C Controller performs an RSTDAA and ENTDAA CCC. If one or more I3C Targets acknowledge the Broadcast Address 0x7E, then the Supernova 
will assign the lower available and valid dynamic address.

In [11]:
# Get the Target Devices Table from the Supernova. It must be empty before the bus initialization.
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 8,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 0,
    "table": []
}


In [14]:
# Initialize and scan the I3C bus.
request_result = supernova.i3cInitBus(getId())

>> New message from SUPERNOVA:
{'id': 11, 'command': 2, 'name': 'I3C INIT BUS', 'result': 'I3C_BUS_INIT_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


In [15]:
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 12,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 1,
    "table": [
        {
            "staticAddress": 0,
            "dynamicAddress": 8,
            "bcr": {
                "value": [
                    "0b00000111",
                    7,
                    "0x07"
                ],
                "description": {
                    "deviceRole": "I3C Target.",
                    "advancedCapabilities": "Does not support optional advanced capabilities.",
                    "virtualTargetSupport": "Is not a Virtual Target and does not expose other downstream Device(s).",
                    "offlineCapable": "Device retains the Dynamic Address and will always respond to I3C Bus commands.",
                    "ibiPayload": "One data byte (MDB) shall follow the accepted IBI, and additional data bytes may follow.",
                    "ibiRequestCapable": "Capable.",
                    "maxData

2. Reset I3C BUS

Let's reset the bus to initialize the bus again, this time sending a non-empty target devices table to the Supernova.

In [ ]:
I3C_BROADCAST_ADDRESS = 0x7E

# Reset the bus issuing the RSTDAA command and clear the target device table.
request_result = supernova.i3cClearFeature(getId(), I3cClearFeatureSelector.I3C_BUS, I3C_BROADCAST_ADDRESS)

In [ ]:
# The target devices table must be empty again after resetting the bus.
request_result = supernova.i3cGetTargetDeviceTable(getId())

3. Add targets to the target device table before initializing the bus using a Python dictionary

In [ ]:
'''
If the static address of a target is known, send it to the Supernova. The static address is mandatory
if it is expected to assign the dynamic address of the target through SETDASA or SETAASA CCCs.
'''
LSMD6V_STATIC_ADDRESS = 0x6B        #ST LSM6DSV Static Address

'''
Select the dynamic address to be assigned to the I3C Target.
'''
LSMD6V_DYNAMIC_ADDRESS = 0x0A

'''
Set the features and the behavior of the I3C Target. For instance, in case that the I3C Target BCR[1] is
set, which means that the I3C Target support In-Band Interrupt (IBI) requests, define whether the Supernova
must accept or reject IBIs request from the target. Define also if the dynamic address will be assigned
using ENTDAA, SETDASA or SETAASA. To assign the dynamic address set above using ENTDAA, it is mandatory
to set also the BCR, DCR and PID in the table. If SETDASA or SETAASA are used, it is mandatory to set
the static address. It is also important to add here the I2C devices, setting the static address and the
target type as I2C_DEVICE.

In the example below, the SETDASA CCC is used to set the dynamic address set above to the target.
'''
LSMD6V_FEATURES =  TargetInterruptRequest.ACCEPT_INTERRUPT_REQUEST.value |  \
                    ControllerRoleRequest.REJECT_CRR.value |   \
                    IBiTimestamp.DISABLE_IBIT.value |   \
                    AssignmentFromStaticAddress.I3C_TARGET_SUPPORTS_SETDASA.value |   \
                    DAaWithENTDAA.DISABLE_ENTDAA.value |   \
                    TargetType.I3C_DEVICE.value |   \
                    PendingReadCapability.SUPPORT_IBI_READ_CAPABILITY.value |   \
                    ValidPID.HAS_VALID_PID.value

'''
In the example below, the ENTDAA CCC is used to set the target dynamic address. To identify the target
is mandatory to set the PID, BCR and DCR.

'''
BMI323_DYNAMIC_ADDRESS = 0x0B

BMI323_PID = [0x07, 0x70, 0x10, 0x43, 0x10, 0x00]
BMI323_BCR = 0x06
BMI323_DCR = 0xEF

BMI323_FEATURES =  TargetInterruptRequest.ACCEPT_INTERRUPT_REQUEST.value |  \
                    ControllerRoleRequest.REJECT_CRR.value |   \
                    IBiTimestamp.DISABLE_IBIT.value |   \
                    AssignmentFromStaticAddress.I3C_TARGET_SUPPORTS_SETDASA |   \
                    DAaWithENTDAA.ENABLE_ENTDAA.value |   \
                    TargetType.I3C_DEVICE.value |   \
                    PendingReadCapability.SUPPORT_IBI_READ_CAPABILITY.value |   \
                    ValidPID.HAS_VALID_PID.value


'''
Add an I2C Device too. In this case, it can be the Sparkfun EEPROM breakout board or the Adafruit I2C FRAM memory.
'''
I2C_STATIC_ADDRESS = 0x50

I2C_FEATURES =  TargetInterruptRequest.REJECT_INTERRUPT_REQUEST.value |  \
                ControllerRoleRequest.REJECT_CRR.value |   \
                IBiTimestamp.DISABLE_IBIT.value |   \
                AssignmentFromStaticAddress.I3C_TARGET_DOES_NOT_HAVE_STATIC_ADDR.value |   \
                DAaWithENTDAA.DISABLE_ENTDAA.value |   \
                TargetType.I2C_DEVICE.value |   \
                PendingReadCapability.NOT_SUPPORT_IBI_READ_CAPABILITY.value |   \
                ValidPID.HAS_NOT_VALID_PID.value

'''
Create the target devices table to be transferred to the Supernova. This table is a dictionary whose keys
are integer numbers starting from 0 and the maximum value possible is 10, which means that the maximum number
of I3C Targets is 11.
'''
table = {0: {'staticAddress': LSMD6V_STATIC_ADDRESS,
        'dynamicAddress': LSMD6V_DYNAMIC_ADDRESS,
        'bcr': 0x00,
        'dcr': 0x00,
        'pid': [0x00, 0x00, 0x00, 0x00, 0x00, 0x00],
        'maxIbiPayloadLength': 9,
        'i3cFeatures': LSMD6V_FEATURES},
        1: {'staticAddress': 0x00,
        'dynamicAddress': BMI323_DYNAMIC_ADDRESS,
        'bcr': BMI323_BCR,
        'dcr': BMI323_DCR,
        'pid': BMI323_PID,
        'maxIbiPayloadLength': 0,
        'i3cFeatures': BMI323_FEATURES},
        2: {'staticAddress': I2C_STATIC_ADDRESS,
        'dynamicAddress': 0x00,
        'bcr': 0x00,
        'dcr': 0x00,
        'pid': [0x00, 0x00, 0x00, 0x00, 0x00, 0x00],
        'maxIbiPayloadLength': 0,
        'i3cFeatures': I2C_FEATURES}
        }

'''
Initialize the bus using the predefined target devices table.
'''
request_result = supernova.i3cInitBus(getId(), table)

In [ ]:
# Get the update target devices table.
request_result = supernova.i3cGetTargetDeviceTable(getId())

### 4. I3C Transfers

#### 4.1. I2C Legacy over I3C

The Supernova, as stated in the MIPI I3C specification, supports backward compatibility with I2C. When an I2C device is connected to the bus,
the device must be added to the I3C Target Devices Table as explained above in section 3.3. Then the Supernova API allows to issue I2C Write,
I2C Read and I2C Read From (I2C Write + RS + I3C Read) transfers.

In [27]:
ID              = getId()
TARGET_ADDR     = I2C_STATIC_ADDRESS
MODE            = TransferMode.I2C_MODE
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_3_75_MHZ    # This value is ignored.
OPEN_DRAIN_RATE = I2cTransferRate._1MHz
REG_ADDR        = [0x00, 0x00]
DATA            = [i for i in range(10)]

# API I3C Read
request_result = supernova.i3cWrite(ID, TARGET_ADDR, MODE, PUSH_PULL_RATE, OPEN_DRAIN_RATE, REG_ADDR, DATA)

>> New message from SUPERNOVA:
{
    "id": 18,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


In [28]:
ID              = getId()
TARGET_ADDR     = I2C_STATIC_ADDRESS
MODE            = TransferMode.I2C_MODE
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ           # This value is ignored.
OPEN_DRAIN_RATE = I2cTransferRate._400KHz
REG_ADDR        = [0x00,0x00]                                       # If this list is empty [], then only and I2C Read transfer is issued.
LEN             = 5

# API I3C Read
request_result = supernova.i3cRead(ID, TARGET_ADDR, MODE, PUSH_PULL_RATE, OPEN_DRAIN_RATE, REG_ADDR, LEN)

>> New message from SUPERNOVA:
{
    "id": 19,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 5,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        1,
        2,
        3,
        4
    ]
}


#### 4.2 Common command code

##### 4.2.0 I3C Set Feature

This command performs the Broadcast or Direct ENEC CCCs based in the dynamic address set in the 3rd parameter. If address is the I3C Broadcast Address 0x7E, then the Boradcast ENEC CCC is issued, otherwise, the Direct ENEC CCC is issued. Then, the second parameter identifies the events that are enabled in the bus. The enum ``I3cSetFeatureSelector`` identifies the diferent events, and the second parameter can be an OR operation between them, for instance `I3cSetFeatureSelector.REGULAR_IBI | I3cSetFeatureSelector.HOT_JOIN`

```python
I3cSetFeatureSelector.REGULAR_IBI
I3cSetFeatureSelector.HOT_JOIN
I3cSetFeatureSelector.I3C_CONTROLLER_ROLE_HANDOFF
```

In [20]:
TARGET_ADDR = BMI323_DYNAMIC_ADDRESS
BROADCAST_ADDRESS = 0x7E

request_result = supernova.i3cSetFeature(getId(), I3cSetFeatureSelector.REGULAR_IBI, BROADCAST_ADDRESS)

>> New message from SUPERNOVA:
{'id': 17, 'command': 8, 'name': 'I3C SET FEATURE', 'result': 'I3C_SET_FEATURE_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


##### 4.2.1 GETPID

In [16]:
TARGET_ADDR = 0x08

request_result = supernova.i3cGETPID(getId(), 
                                    TARGET_ADDR, 
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ,
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ)

>> New message from SUPERNOVA:
{
    "id": 13,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 6,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "pid": [
        "0x02",
        "0x08",
        "0x00",
        "0x70",
        "0x92",
        "0x0b"
    ]
}


##### 4.2.2 GETBCR

In [22]:
TARGET_ADDR = 0x08

request_result = supernova.i3cGETBCR(getId(), 
                                    TARGET_ADDR, 
                                    I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ,
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_2_5_MHZ)

>> New message from SUPERNOVA:
{
    "id": 19,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "bcr": {
        "value": [
            "0b00100111",
            39,
            "0x27"
        ],
        "description": {
            "deviceRole": "I3C Target.",
            "advancedCapabilities": "Supports optional advanced capabilities. Use GETCAPS CCC to deremine which ones.",
            "virtualTargetSupport": "Is not a Virtual Target and does not expose other downstream Device(s).",
            "offlineCapable": "Device retains the Dynamic Address and will always respond to I3C Bus commands.",
            "ibiPayload": "One data byte (MDB) shall follow the accepted IBI, and additional data bytes may follow.",
            "ibiRequestCa

##### 4.2.3 GETDCR

In [23]:
TARGET_ADDR = 0x0A

request_result = supernova.i3cGETDCR(getId(), 
                                    TARGET_ADDR, 
                                    I3cPushPullTransferRate.PUSH_PULL_5_MHZ,
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ)

>> New message from SUPERNOVA:
{
    "id": 20,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "dcr": "0xa0"
}


##### 4.2.4 GETCAPS

In [24]:
TARGET_ADDR = 0x08

request_result = supernova.i3cGETCAPS(getId(), 
                                    TARGET_ADDR, 
                                    I3cPushPullTransferRate.PUSH_PULL_5_MHZ,
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ)

>> New message from SUPERNOVA:
{
    "id": 21,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "caps1": {
        "value": [
            "0b00000001",
            1,
            "0x01"
        ],
        "description": {
            "hdrMode0": "HDR Mode 0 (HDR-DDR) supported",
            "hdrMode1": "HDR Mode 1 not supported (Not included in I3C Basic)",
            "hdrMode2": "HDR Mode 2 not supported (Not included in I3C Basic)",
            "hdrMode3": "HDR Mode 3 (HDR-BT) not supported",
            "hdrMode4": "Reserved",
            "hdrMode5": "Reserved",
            "hdrMode6": "Reserved",
            "hdrMode7": "Reserved"
        }
    },
    "caps2": {
        "value": [
            "0b00000000",
            0,
            "

##### 4.2.5 GETMXDS

In [25]:
TARGET_ADDR = 0x08

request_result = supernova.i3cGETMXDS(getId(), 
                                    TARGET_ADDR, 
                                    I3cPushPullTransferRate.PUSH_PULL_5_MHZ,
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ)

>> New message from SUPERNOVA:
{
    "id": 22,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "maxWr": {
        "value": [
            "0b00000000",
            0,
            "0x00"
        ],
        "description": {
            "maxSustainedDataRate": "fSCL Max (default value)",
            "definingByteSupport": "Defining byte not supported",
            "reserved": "Reserved for future use by MIPI Alliance"
        }
    },
    "maxRd": {
        "value": [
            "0b00111000",
            56,
            "0x38"
        ],
        "description": {
            "maxSustainedDataRate": "fSCL Max (default value)",
            "clockToDataTurnaroundTime": "Tsco is > 12 ns, and is reported by private agreement",
            "writeTo

##### 4.2.6 GETMRL

In [26]:
TARGET_ADDR = 0x08

request_result = supernova.i3cGETMRL(getId(),
                                    TARGET_ADDR,
                                    I3cPushPullTransferRate.PUSH_PULL_5_MHZ,
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ)

>> New message from SUPERNOVA:
{
    "id": 42,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 3,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "maxReadLength": 34,
    "maxIbiPayloadSize": 7
}


##### 4.2.7 Direct SETMRL

In [44]:
TARGET_ADDR = 0x08
MAXIMUM_READ_LEN = 0x22
IBI_PAYLOAD_SIZE = 0x07

request_result = supernova.i3cDirectSETMRL(getId(),
                                            TARGET_ADDR,
                                            I3cPushPullTransferRate.PUSH_PULL_5_MHZ,
                                            I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ,
                                            MAXIMUM_READ_LEN,
                                            IBI_PAYLOAD_SIZE)

>> New message from SUPERNOVA:
{
    "id": 24,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        0
    ]
}


##### 4.2.8 Braodcast SETMRL

In [40]:
MAXIMUM_READ_LEN = 0x0052
IBI_PAYLOAD_SIZE = 0x09

request_result = supernova.i3cBroadcastSETMRL(getId(),
                                            I3cPushPullTransferRate.PUSH_PULL_5_MHZ,
                                            I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ,
                                            MAXIMUM_READ_LEN,
                                            IBI_PAYLOAD_SIZE)

>> New message from SUPERNOVA:
{
    "id": 37,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 3,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        0,
        0
    ]
}


##### 4.2.9 GETMWL

In [35]:
TARGET_ADDR = 0x08

request_result = supernova.i3cGETMWL(getId(),
                                    TARGET_ADDR,
                                    I3cPushPullTransferRate.PUSH_PULL_5_MHZ,
                                    I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ)

>> New message from SUPERNOVA:
{
    "id": 32,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "maxWriteLength": 256
}


##### 4.2.10 Direct SETMWL

In [32]:
TARGET_ADDR = 0x08
MAXIMUM_WRITE_LEN = 22

request_result = supernova.i3cDirectSETMWL(getId(),
                                            TARGET_ADDR,
                                            I3cPushPullTransferRate.PUSH_PULL_5_MHZ,
                                            I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ,
                                            MAXIMUM_WRITE_LEN)

>> New message from SUPERNOVA:
{
    "id": 29,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        0
    ]
}


##### 4.2.11 Broadcast SETMWL

In [34]:
MAXIMUM_WRITE_LEN = 256

request_result = supernova.i3cBroadcastSETMWL(getId(),
                                                I3cPushPullTransferRate.PUSH_PULL_5_MHZ,
                                                I3cOpenDrainTransferRate.OPEN_DRAIN_1_25_MHZ,
                                                MAXIMUM_WRITE_LEN)

>> New message from SUPERNOVA:
{
    "id": 31,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 3,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        0,
        0
    ]
}


#### 4.3 Private Write/Read Transfers

#### 4.3.1 Private Read Transfers

In [32]:
TARGET_ADDR     = 0x0A      # LSM6DSV dynamic address
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ
REG_ADDR        = [0x0F]    # WHO_AM_I register: 0x70 = 112 expected.
LEN             = 1

# API I3C Read
request_result = supernova.i3cRead(getId(), TARGET_ADDR, MODE, PUSH_PULL_RATE, OPEN_DRAIN_RATE, REG_ADDR, LEN)

>> New message from SUPERNOVA:
{
    "id": 29,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        66
    ]
}


#### 4.3.2 Private Write Transfers

In [33]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_12_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ
REG_ADDR        = [0x16]
DATA            = [0x40]        # Python list of bytes. Maximum length of 1024 bytes.

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from SUPERNOVA:
{
    "id": 30,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


### 5. Change dynamic address

This method allows the user to change the dynamic address to an I3C Target. In this case, the Supernova as the I3C Controller issues the CCC SETNEWDA. The new address will be updated in the target devices table if the target acknowledges it.

In [34]:
TARGET_ADDR     = 0x08
NEW_ADDR        = 0x0A

request_result = supernova.i3cChangeDynamicAddress(getId(), TARGET_ADDR, NEW_ADDR)

# IMPORTANT: Be careful and use the new address after running this command.

>> New message from SUPERNOVA:
{'id': 31, 'command': 5, 'name': 'I3C CHANGE DA', 'result': 'I3C_CHANGE_DYNAMIC_ADDRESS_SUCCESS', 'errors': ['NO_TRANSFER_ERROR']}


### 6. I3C Target Reset

The Supernova as the I3C Controller provides support to the I3C Target Reset Pattern. The SUpernova class provides methods to support the RSTACT CCC and a separated method to trigger the I3C Target Reset Pattern into the bus.

##### 6.1 Direct RSTACT in read mode

In [35]:
TARGET_ADDR     = 0x0A
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ

request_result = supernova.i3cDirectRSTACT(getId(), TARGET_ADDR, I3cTargetResetDefByte.RESET_I3C_PERIPHERAL, TransferDirection.READ, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from SUPERNOVA:
{
    "id": 32,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 1,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0
    ]
}


##### 6.2 Direct RSTACT in write mode

In [36]:
TARGET_ADDR     = 0x0A
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ

request_result = supernova.i3cDirectRSTACT(getId(), TARGET_ADDR, I3cTargetResetDefByte.RESET_I3C_PERIPHERAL, TransferDirection.WRITE, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from SUPERNOVA:
{
    "id": 33,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


##### 6.3 Trigger the Target Reset Pattern

In [37]:
request_result = supernova.i3cTriggerTargetResetPattern(getId())

>> New message from SUPERNOVA:
{'id': 34, 'command': 25, 'name': 'I3C TRIGGER TARGET RESET PATTERN', 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_CONTROLLER_MGR_NO_ERROR', 'driver_result': ['NO_TRANSFER_ERROR']}


##### 6.4 Reset all the whole Targets using Broadcast RSTACT CCC.

In [38]:
TARGET_ADDR     = 0x0A
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ

request_result = supernova.i3cBroadcastRSTACT(getId(), I3cTargetResetDefByte.RESET_WHOLE_TARGET, PUSH_PULL_RATE, OPEN_DRAIN_RATE)

>> New message from SUPERNOVA:
{
    "id": 35,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": true
    },
    "descriptor": {
        "dataLength": 2,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": [
        0,
        0
    ]
}


In [39]:
request_result = supernova.i3cTriggerTargetResetPattern(getId())

>> New message from SUPERNOVA:
{'id': 36, 'command': 25, 'name': 'I3C TRIGGER TARGET RESET PATTERN', 'usb_result': 'CMD_SUCCESSFUL', 'manager_result': 'I3C_CONTROLLER_MGR_NO_ERROR', 'driver_result': ['NO_TRANSFER_ERROR']}


### 7. Close communication

Use the ``Supernova.close()`` method to end the communication with the Supernova device and release the used memory in the background like threads and so on.

In [40]:
# Close the communication with the Supernova device.
supernova.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}